# ALL IMPORTS NECESSARY ARE HERE AT THE TOP

In [ ]:
import scipy.io as sio
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from google.colab import drive
from scipy.interpolate import interp1d
import random
from keras.models import Sequential
from keras.layers import Dense
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
!pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.1
    Uninstalling numpy-1.25.1:
      Successfully uninstalled numpy-1.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.
flax 0.7.0 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
orbax-checkpoint 0.2.7 requires jax>=0.4.9, but you have jax 0.3.25 which is incompatible.


https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall?hl=pt-br

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# READING THE DATA

In [ ]:
file_paths = [
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario1.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario2.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario3.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario4.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario5.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario6.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario7.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario8.mat',
    '/content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario9.mat'
]

file_data = []
df = []

for i, file_path in enumerate(file_paths):
    print(f"Reading File: {file_path}")
    data = sio.loadmat(file_path)
    df.append(pd.DataFrame(data['data'], columns=['ALE', 'NLE', 'ALD', 'NLD','S4_y1', 'S4_z1', 'S1_y1', 'S1_z1', 'S2_y1', 'S2_z1', 'S3_y1', 'S3_z1', 'Label']))

Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario1.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario2.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario3.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario4.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario5.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario6.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario7.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario8.mat
Reading File: /content/drive/MyDrive/Tese_Dados/Final_Data/final_data_damaged_scenario9.mat


#NN Regressor

#1%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:10000]
dataset2 = dataset2[:10000]
dataset3 = dataset3[:10000]
dataset4 = dataset4[:10000]
dataset5 = dataset5[:10000]
dataset6 = dataset6[:10000]
dataset7 = dataset7[:10000]
dataset8 = dataset8[:10000]
dataset9 = dataset9[:10000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - Neural Network

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a neural network model for irregularity prediction
irregularity_model = Sequential()
irregularity_model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(4))  # 4 output neurons for irregularity columns

irregularity_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
irregularity_model.fit(X_train_scaled, y_train, epochs=50)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 1%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

Epoch 1/50
2532/2532 [==============================] - 22s 5ms/step - loss: 0.8878 - accuracy: 0.3169
Epoch 2/50
2532/2532 [==============================] - 13s 5ms/step - loss: 0.8568 - accuracy: 0.3388
Epoch 3/50
2532/2532 [==============================] - 13s 5ms/step - loss: 0.8165 - accuracy: 0.3635
Epoch 4/50
2532/2532 [==============================] - 15s 6ms/step - loss: 0.7980 - accuracy: 0.3861
Epoch 5/50
2532/2532 [==============================] - 13s 5ms/step - loss: 0.7875 - accuracy: 0.3939
Epoch 6/50
2532/2532 [==============================] - 13s 5ms/step - loss: 0.7743 - accuracy: 0.3987
Epoch 7/50
2532/2532 [==============================] - 13s 5ms/step - loss: 0.7653 - accuracy: 0.4075
Epoch 8/50
2532/2532 [==============================] - 13s 5ms/step - loss: 0.7574 - accuracy: 0.4126
Epoch 9/50
2532/2532 [==============================] - 13s 5ms/step - loss: 0.7530 - accuracy: 0.4169
Epoch 10/50
2532/2532 [==============================] - 13s 5ms/step - l

Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

282/282 [==============================] - 0s 2ms/step
Evaluation Metrics:
Precision: 0.2769
Recall: 0.2981
F1-score: 0.2871
Accuracy: 0.6326
True Positives (TP): 666
False Positives (FP): 1739
True Negatives (TN): 5027
False Negatives (FN): 1568


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
2532/2532 [==============================] - 10s 3ms/step - loss: 0.4586 - accuracy: 0.7911
Epoch 2/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4495 - accuracy: 0.7957
Epoch 3/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4464 - accuracy: 0.7981
Epoch 4/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4440 - accuracy: 0.7989
Epoch 5/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4419 - accuracy: 0.8004
Epoch 6/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4404 - accuracy: 0.8015
Epoch 7/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4390 - accuracy: 0.8016
Epoch 8/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4374 - accuracy: 0.8025
Epoch 9/10
2532/2532 [==============================] - 8s 3ms/step - loss: 0.4359 - accuracy: 0.8038
Epoch 10/10
282/282 [==============================] - 0s 1ms/step
Evaluation Met

Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

282/282 [==============================] - 0s 2ms/step
Evaluation Metrics:
Precision: 0.2855
Recall: 0.3227
F1-score: 0.3030
Accuracy: 0.6314
True Positives (TP): 721
False Positives (FP): 1804
True Negatives (TN): 4962
False Negatives (FN): 1513


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

282/282 [==============================] - 1s 2ms/step
Evaluation Metrics:
Precision: 0.2896
Recall: 0.2153
F1-score: 0.2470
Accuracy: 0.6741
True Positives (TP): 481
False Positives (FP): 1180
True Negatives (TN): 5586
False Negatives (FN): 1753


Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

282/282 [==============================] - 1s 2ms/step
Evaluation Metrics:
Precision: 0.2482
Recall: 1.0000
F1-score: 0.3977
Accuracy: 0.2482
True Positives (TP): 2234
False Positives (FP): 6766
True Negatives (TN): 0
False Negatives (FN): 0


#5%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:50000]
dataset2 = dataset2[:50000]
dataset3 = dataset3[:50000]
dataset4 = dataset4[:50000]
dataset5 = dataset5[:50000]
dataset6 = dataset6[:50000]
dataset7 = dataset7[:50000]
dataset8 = dataset8[:50000]
dataset9 = dataset9[:50000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - RANDOM FOREST

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a neural network model for irregularity prediction
irregularity_model = Sequential()
irregularity_model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(4))  # 4 output neurons for irregularity columns

irregularity_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
irregularity_model.fit(X_train_scaled, y_train, epochs=50)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 5%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

Epoch 1/50
12657/12657 [==============================] - 72s 5ms/step - loss: 0.9426 - accuracy: 0.2842
Epoch 2/50
12657/12657 [==============================] - 68s 5ms/step - loss: 0.9303 - accuracy: 0.2867
Epoch 3/50
12657/12657 [==============================] - 71s 6ms/step - loss: 0.9219 - accuracy: 0.2883
Epoch 4/50
12657/12657 [==============================] - 67s 5ms/step - loss: 0.9151 - accuracy: 0.2891
Epoch 5/50
12657/12657 [==============================] - 69s 5ms/step - loss: 0.9099 - accuracy: 0.2916
Epoch 6/50
12657/12657 [==============================] - 69s 5ms/step - loss: 0.9060 - accuracy: 0.2918
Epoch 7/50
12657/12657 [==============================] - 68s 5ms/step - loss: 0.9016 - accuracy: 0.2925
Epoch 8/50
12657/12657 [==============================] - 68s 5ms/step - loss: 0.8983 - accuracy: 0.2930
Epoch 9/50
12657/12657 [==============================] - 67s 5ms/step - loss: 0.8939 - accuracy: 0.2964
Epoch 10/50
12657/12657 [==============================

Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

1407/1407 [==============================] - 2s 2ms/step
Evaluation Metrics:
Precision: 0.3447
Recall: 0.3710
F1-score: 0.3574
Accuracy: 0.5442
True Positives (TP): 5703
False Positives (FP): 10840
True Negatives (TN): 18788
False Negatives (FN): 9669


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
12657/12657 [==============================] - 43s 3ms/step - loss: 0.5964 - accuracy: 0.6936
Epoch 2/10
12657/12657 [==============================] - 42s 3ms/step - loss: 0.5905 - accuracy: 0.6986
Epoch 3/10
12657/12657 [==============================] - 42s 3ms/step - loss: 0.5880 - accuracy: 0.7002
Epoch 4/10
12657/12657 [==============================] - 42s 3ms/step - loss: 0.5861 - accuracy: 0.7019
Epoch 5/10
12657/12657 [==============================] - 43s 3ms/step - loss: 0.5846 - accuracy: 0.7029
Epoch 6/10
12657/12657 [==============================] - 42s 3ms/step - loss: 0.5834 - accuracy: 0.7041
Epoch 7/10
12657/12657 [==============================] - 42s 3ms/step - loss: 0.5822 - accuracy: 0.7050
Epoch 8/10
12657/12657 [==============================] - 43s 3ms/step - loss: 0.5816 - accuracy: 0.7057
Epoch 9/10
12657/12657 [==============================] - 42s 3ms/step - loss: 0.5808 - accuracy: 0.7063
Epoch 10/10
1407/1407 [==============================] 

Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

1407/1407 [==============================] - 2s 2ms/step
Evaluation Metrics:
Precision: 0.3589
Recall: 0.4385
F1-score: 0.3947
Accuracy: 0.5406
True Positives (TP): 6741
False Positives (FP): 12042
True Negatives (TN): 17586
False Negatives (FN): 8631


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

1407/1407 [==============================] - 2s 2ms/step
Evaluation Metrics:
Precision: 0.3533
Recall: 0.5086
F1-score: 0.4170
Accuracy: 0.5141
True Positives (TP): 7818
False Positives (FP): 14310
True Negatives (TN): 15318
False Negatives (FN): 7554


Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

1407/1407 [==============================] - 3s 2ms/step
Evaluation Metrics:
Precision: 0.4080
Recall: 0.7325
F1-score: 0.5240
Accuracy: 0.5455
True Positives (TP): 11260
False Positives (FP): 16341
True Negatives (TN): 13287
False Negatives (FN): 4112


#25%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:250000]
dataset2 = dataset2[:250000]
dataset3 = dataset3[:250000]
dataset4 = dataset4[:250000]
dataset5 = dataset5[:250000]
dataset6 = dataset6[:250000]
dataset7 = dataset7[:250000]
dataset8 = dataset8[:250000]
dataset9 = dataset9[:250000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - RANDOM FOREST

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a neural network model for irregularity prediction
irregularity_model = Sequential()
irregularity_model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(4))  # 4 output neurons for irregularity columns

irregularity_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
irregularity_model.fit(X_train_scaled, y_train, epochs=25)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 25%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

# Step 2: Train an SVC model for damage prediction
X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

Epoch 1/25
63282/63282 [==============================] - 206s 3ms/step - loss: 0.9217 - accuracy: 0.3560
Epoch 2/25
63282/63282 [==============================] - 201s 3ms/step - loss: 0.9127 - accuracy: 0.3616
Epoch 3/25
63282/63282 [==============================] - 200s 3ms/step - loss: 0.9100 - accuracy: 0.3622
Epoch 4/25
63282/63282 [==============================] - 198s 3ms/step - loss: 0.9085 - accuracy: 0.3622
Epoch 5/25
63282/63282 [==============================] - 199s 3ms/step - loss: 0.9074 - accuracy: 0.3623
Epoch 6/25
63282/63282 [==============================] - 200s 3ms/step - loss: 0.9068 - accuracy: 0.3623
Epoch 7/25
63282/63282 [==============================] - 198s 3ms/step - loss: 0.9062 - accuracy: 0.3620
Epoch 8/25
63282/63282 [==============================] - 197s 3ms/step - loss: 0.9057 - accuracy: 0.3614
Epoch 9/25
63282/63282 [==============================] - 199s 3ms/step - loss: 0.9056 - accuracy: 0.3614
Epoch 10/25
63282/63282 [=====================

Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

7032/7032 [==============================] - 10s 1ms/step
Evaluation Metrics:
Precision: 0.4215
Recall: 0.5093
F1-score: 0.4613
Accuracy: 0.5209
True Positives (TP): 46153
False Positives (FP): 63337
True Negatives (TN): 71049
False Negatives (FN): 44461


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
63282/63282 [==============================] - 212s 3ms/step - loss: 0.6340 - accuracy: 0.6478
Epoch 2/10
63282/63282 [==============================] - 205s 3ms/step - loss: 0.6313 - accuracy: 0.6505
Epoch 3/10
63282/63282 [==============================] - 211s 3ms/step - loss: 0.6302 - accuracy: 0.6519
Epoch 4/10
63282/63282 [==============================] - 209s 3ms/step - loss: 0.6294 - accuracy: 0.6527
Epoch 5/10
63282/63282 [==============================] - 209s 3ms/step - loss: 0.6289 - accuracy: 0.6531
Epoch 6/10
63282/63282 [==============================] - 211s 3ms/step - loss: 0.6284 - accuracy: 0.6533
Epoch 7/10
63282/63282 [==============================] - 214s 3ms/step - loss: 0.6281 - accuracy: 0.6539
Epoch 8/10
63282/63282 [==============================] - 207s 3ms/step - loss: 0.6278 - accuracy: 0.6542
Epoch 9/10
63282/63282 [==============================] - 208s 3ms/step - loss: 0.6275 - accuracy: 0.6543
Epoch 10/10
7032/7032 [=======================

Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

7032/7032 [==============================] - 10s 1ms/step
Evaluation Metrics:
Precision: 0.4213
Recall: 0.5960
F1-score: 0.4937
Accuracy: 0.5077
True Positives (TP): 54009
False Positives (FP): 74172
True Negatives (TN): 60214
False Negatives (FN): 36605


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

7032/7032 [==============================] - 10s 1ms/step
Evaluation Metrics:
Precision: 0.4164
Recall: 0.6045
F1-score: 0.4931
Accuracy: 0.4994
True Positives (TP): 54779
False Positives (FP): 76789
True Negatives (TN): 57597
False Negatives (FN): 35835


Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

7032/7032 [==============================] - 9s 1ms/step
Evaluation Metrics:
Precision: 0.4680
Recall: 0.7597
F1-score: 0.5792
Accuracy: 0.5554
True Positives (TP): 68842
False Positives (FP): 78253
True Negatives (TN): 56133
False Negatives (FN): 21772


#100%

In [ ]:
dataset1 = df[0]
dataset2 = df[1]
dataset3 = df[2]
dataset4 = df[3]
dataset5 = df[4]
dataset6 = df[5]
dataset7 = df[6]
dataset8 = df[7]
dataset9 = df[8]

dataset1 = dataset1[:1000000]
dataset2 = dataset2[:1000000]
dataset3 = dataset3[:1000000]
dataset4 = dataset4[:1000000]
dataset5 = dataset5[:1000000]
dataset6 = dataset6[:1000000]
dataset7 = dataset7[:1000000]
dataset8 = dataset8[:1000000]
dataset9 = dataset9[:1000000]

print("Datasets read successfully")

Datasets read successfully


TRAIN STEP 1 REGRESSOR - RANDOM FOREST

In [ ]:
# Concatenate datasets 1, 2, 3, 4, 5, 7, 8, and 9 for training
#train_datasets = [dataset1_sample, dataset2_sample, dataset3_sample, dataset4_sample, dataset5_sample, dataset7_sample, dataset8_sample, dataset9_sample]

train_datasets = [dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7, dataset8, dataset9]

train_data = pd.concat(train_datasets).reset_index(drop=True)

# Separate the training data into features (X) and irregularity data (y)
X = train_data.iloc[:, 4:-1]  # Only acceleration data
y = train_data[['ALE', 'NLE', 'ALD', 'NLD']]  # Irregularity data
z = train_data.iloc[:, -1]

# Create the scaler and scale the training features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train_scaled, X_test_scaled, y_train, y_test, z_train, z_test = train_test_split(X_scaled, y, z, test_size=0.1, random_state=42)

# Step 1: Train a neural network model for irregularity prediction
irregularity_model = Sequential()
irregularity_model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(64, activation='relu'))
irregularity_model.add(Dense(4))  # 4 output neurons for irregularity columns

irregularity_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
irregularity_model.fit(X_train_scaled, y_train, epochs=15)

# Predict irregularity for the training data
irregularity_predictions = irregularity_model.predict(X_train_scaled)

# Calculate evaluation metrics for Step 1
mse = mean_squared_error(y_train, irregularity_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_train, irregularity_predictions)
r2 = r2_score(y_train, irregularity_predictions)

# Print the evaluation metrics for Step 1
print("Metrics for the RF Regressor - 100%")
print("RMSE:", rmse)
print("R2:", r2)
print("MAE:", mae)

# Step 2: Train an SVC model for damage prediction
X_train_combined = np.column_stack((X_train_scaled, irregularity_predictions))

Epoch 1/15
253125/253125 [==============================] - 361s 1ms/step - loss: 0.9055
Epoch 2/15
253125/253125 [==============================] - 359s 1ms/step - loss: 0.9011
Epoch 3/15
253125/253125 [==============================] - 353s 1ms/step - loss: 0.9001
Epoch 4/15
253125/253125 [==============================] - 342s 1ms/step - loss: 0.8995
Epoch 5/15
253125/253125 [==============================] - 327s 1ms/step - loss: 0.8992
Epoch 6/15
253125/253125 [==============================] - 322s 1ms/step - loss: 0.8989
Epoch 7/15
253125/253125 [==============================] - 341s 1ms/step - loss: 0.8988
Epoch 8/15
253125/253125 [==============================] - 330s 1ms/step - loss: 0.8987
Epoch 9/15
253125/253125 [==============================] - 353s 1ms/step - loss: 0.8985
Epoch 10/15
253125/253125 [==============================] - 373s 1ms/step - loss: 0.8984
Epoch 11/15
253125/253125 [==============================] - 353s 1ms/step - loss: 0.8983
Epoch 12/15
253125/

Classifier Thresholds

In [ ]:
# Predict irregularity for the testing data
irregularity_predictions = irregularity_model.predict(X_test_scaled)

# Compare the predicted irregularities with a threshold to predict damage
threshold_ale = 1.5e-3
threshold_nle = 2.3e-3
threshold_ald = 1.5e-3
threshold_nld = 2.3e-3

predicted_labels = []
for i in range(len(irregularity_predictions)):
    if (irregularity_predictions[i, 0] > threshold_ale) or (irregularity_predictions[i, 1] > threshold_nle) or (irregularity_predictions[i, 2] > threshold_ald) or (irregularity_predictions[i, 3] > threshold_nld):
        predicted_labels.append(1)
    else:
        predicted_labels.append(0)

# Extract the corresponding labels from the original dataset using the index of the test data
actual_labels = z_test.values

# Calculate evaluation metrics based on the predicted labels and the actual labels
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
f1 = f1_score(actual_labels, predicted_labels)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, predicted_labels)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

28125/28125 [==============================] - 49s 2ms/step
Evaluation Metrics:
Precision: 0.4992
Recall: 0.7071
F1-score: 0.5853
Accuracy: 0.5908
True Positives (TP): 259844
False Positives (FP): 260626
True Negatives (TN): 271871
False Negatives (FN): 107659


Classifier LightGBM

In [ ]:
# Step 2: Train a LightGBM classifier for damage prediction
damage_model3 = lgb.LGBMClassifier(n_estimators=50, random_state=42)
damage_model3.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model3.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

28125/28125 [==============================] - 40s 1ms/step
Evaluation Metrics:
Precision: 0.4304
Recall: 0.4907
F1-score: 0.4586
Accuracy: 0.5269
True Positives (TP): 180319
False Positives (FP): 238649
True Negatives (TN): 293848
False Negatives (FN): 187184


Classifier NEURAL NETWORK

In [ ]:
# Step 2: Train a neural network model for damage prediction
damage_model5 = Sequential()
damage_model5.add(Dense(64, activation='relu', input_dim=X_train_combined.shape[1]))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(64, activation='relu'))
damage_model5.add(Dense(1, activation='sigmoid'))  # Output neuron for damage classification

damage_model5.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
damage_model5.fit(X_train_combined, z_train, epochs=10, batch_size=32)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model5.predict(X_test_combined)

# Convert damage probabilities to binary predictions using a threshold
threshold = 0.5
damage_predictions = (damage_predictions > threshold).astype(int)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

Epoch 1/10
253125/253125 [==============================] - 535s 2ms/step - loss: 0.6320 - accuracy: 0.6502
Epoch 2/10
253125/253125 [==============================] - 529s 2ms/step - loss: 0.6312 - accuracy: 0.6511
Epoch 3/10
253125/253125 [==============================] - 536s 2ms/step - loss: 0.6310 - accuracy: 0.6514
Epoch 4/10
253125/253125 [==============================] - 549s 2ms/step - loss: 0.6307 - accuracy: 0.6517
Epoch 5/10
253125/253125 [==============================] - 550s 2ms/step - loss: 0.6307 - accuracy: 0.6517
Epoch 6/10
253125/253125 [==============================] - 551s 2ms/step - loss: 0.6305 - accuracy: 0.6519
Epoch 7/10
253125/253125 [==============================] - 549s 2ms/step - loss: 0.6305 - accuracy: 0.6520
Epoch 8/10
253125/253125 [==============================] - 540s 2ms/step - loss: 0.6304 - accuracy: 0.6522
Epoch 9/10
253125/253125 [==============================] - 546s 2ms/step - loss: 0.6303 - accuracy: 0.6523
Epoch 10/10
28125/28125 [===

Classifier XGBoost

In [ ]:
# Step 2: Train an XGBoost classifier for damage prediction
damage_model2 = xgb.XGBClassifier(n_estimators=50, random_state=42)
damage_model2.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model2.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

28125/28125 [==============================] - 44s 2ms/step
Evaluation Metrics:
Precision: 0.4265
Recall: 0.4918
F1-score: 0.4568
Accuracy: 0.5225
True Positives (TP): 180735
False Positives (FP): 243014
True Negatives (TN): 289483
False Negatives (FN): 186768


Classifier RANDOM FOREST

In [ ]:
# Step 2: Train a Random Forest classifier for damage prediction
damage_model6 = RandomForestClassifier(n_estimators=50, random_state=42)
damage_model6.fit(X_train_combined, z_train)

# Step 3: Use the trained models for prediction on dataset6_sample
irregularity_predictions = irregularity_model.predict(X_test_scaled)
X_test_combined = np.column_stack((irregularity_predictions, X_test_scaled))
damage_predictions = damage_model6.predict(X_test_combined)

actual_labels = z_test.values
accuracy = accuracy_score(actual_labels, damage_predictions)
precision = precision_score(actual_labels, damage_predictions)
recall = recall_score(actual_labels, damage_predictions)
f1 = f1_score(actual_labels, damage_predictions)

print("Evaluation Metrics:")
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1))
print("Accuracy: {:.4f}".format(accuracy))

cm = confusion_matrix(actual_labels, damage_predictions)
tn, fp, fn, tp = cm.ravel()

print("True Positives (TP):", tp)
print("False Positives (FP):", fp)
print("True Negatives (TN):", tn)
print("False Negatives (FN):", fn)

28125/28125 [==============================] - 26s 920us/step
Evaluation Metrics:
Precision: 0.4192
Recall: 0.5872
F1-score: 0.4892
Accuracy: 0.4993
True Positives (TP): 215782
False Positives (FP): 298941
True Negatives (TN): 233556
False Negatives (FN): 151721
